In [9]:
import gzip, os
from numpy import *
from pylab import *
from scipy.stats import zscore, pearsonr, linregress, scoreatpercentile
from pandas import DataFrame, date_range, Series, tseries, concat, HDFStore, merge, read_csv, read_pickle, to_datetime 
import pandas as pd
import urllib.request, csv, re,  calendar, time, copy
import _pickle as cPickle
from datetime import datetime,  date, timedelta
import statsmodels.api as sm
import statsmodels.formula.api as smf

import pickle


#### Load in data, create date range, general prep

In [10]:
# This gives a z score
def pd_zscore(x): return (x - mean(x)) / std(x)

# These are going to be binary variables?
DAY_NAMES = ['MON','TUE','WED','THU','FRI','SAT','SUN']
MONTH_NAMES = ['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC ']

DATE_RANGES = {2012: pd.date_range(datetime(2012, 1, 1), datetime(2012, 12,31))}
ANALYSIS_YEAR = 2012
analysis_range = DATE_RANGES[ANALYSIS_YEAR]



In [11]:
# lottery sales
lott = pd.read_csv(r"C:\Users\clark\OneDrive - University of Warwick\Diss\2. Analysis\Data\Otto data\nyc_2012_lottery.csv", index_col= "SalesDate")
# sports pe
sports = pd.read_csv(r"C:\Users\clark\OneDrive - University of Warwick\Diss\2. Analysis\Data\Otto data\Sports\nyc_sports_2012.csv")
# All the unique zip codes
all_zips = lott.ZIP.unique()

#### NOT WORKING: Reading in the demographics.dat file

In [12]:
# Load in the demographics data
demo = r"C:\Users\clark\OneDrive - University of Warwick\Diss\2. Analysis\Data\Otto data\Misc\nyc_demographics.csv"
extended_count_demo = r"C:\Users\clark\OneDrive - University of Warwick\Diss\2. Analysis\Data\Otto data\Misc\extended_county_demographics.csv"

county_demographics = pd.read_csv(extended_count_demo)
demographics = read_csv(demo, index_col = 0)
demographics['z_ses'] = zscore(demographics.ses)

# county_demographics["county"].unique()

In [13]:
demographics

,11,10,aa,12,ma,ba,5+6,pro,phd,sc1,...,workforce,mbsa_workforce,z_mbsa_workforce,education,z_education,ses,year,ZIP,desc,z_ses
10001,0.007731,0.004064,0.033986,0.012903,0.165312,0.293419,0.007072,0.050267,0.023537,0.020502,...,0.716054,0.417786,1.150630,12.005963,1.371467,3.757875,2011.5,10001,Manhattan:Chelsea and Clinton,1.322691
10002,0.020309,0.021815,0.028394,0.023075,0.049255,0.148375,0.060470,0.010675,0.007740,0.016847,...,0.474805,0.267761,-0.109378,8.623016,-0.378415,-0.888709,2011.5,10002,Manhattan:Lower East Side,-0.312806
10003,0.006560,0.004409,0.021294,0.004839,0.182264,0.360839,0.002753,0.080356,0.035317,0.017454,...,0.806745,0.445035,1.379895,13.550556,2.170297,4.960565,2011.5,10003,Manhattan:Lower East Side,1.746011
10004,0.005385,0.000000,0.022193,0.000000,0.169060,0.300751,0.000000,0.059726,0.006364,0.009791,...,0.703655,0.558290,2.334642,10.903068,0.799748,6.796904,2011.5,10004,Manhattan:Lower Manhattan,2.392363
10005,0.000000,0.000211,0.006532,0.001896,0.151145,0.252283,0.000000,0.067636,0.015522,0.004355,...,0.640680,0.515489,1.973114,9.132111,-0.118775,4.140252,2011.5,10005,Manhattan:Lower Manhattan,1.457279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11691,0.028696,0.018618,0.034569,0.026143,0.037358,0.083963,0.021563,0.009956,0.003426,0.022988,...,0.415758,0.206655,-0.622135,7.559812,-0.929046,-2.198012,2011.5,11691,Queens:Rockaways,-0.773653
11692,0.028345,0.013860,0.036001,0.026609,0.020519,0.068786,0.013803,0.010985,0.001195,0.032757,...,0.397091,0.198353,-0.691515,7.408546,-1.006861,-2.301510,2011.5,11692,Queens:Rockaways,-0.810081
11693,0.032669,0.021035,0.039281,0.029621,0.025371,0.099725,0.009359,0.012664,0.004164,0.024599,...,0.408689,0.253407,-0.230642,8.455890,-0.465049,-1.078849,2011.5,11693,Queens:Rockaways,-0.379731
11694,0.012645,0.009170,0.059515,0.013194,0.103851,0.165221,0.009196,0.033807,0.005695,0.042246,...,0.525604,0.304508,0.200771,10.975468,0.838309,1.109041,2011.5,11694,Queens:Rockaways,0.390358


In [14]:
# Creating a lottery jackpot dataframe?
jackpot = pd.DataFrame({'date':analysis_range})
jackpot.index = jackpot.date

#### Create the bad weather regressors

In [15]:
# Dictionary with the dates of hurrican irene and sandy
hurricanes = { 'sandy': pd.date_range(datetime(2012,10,29),datetime(2012,11,1)) }

# Create empty regressor pandas
storm_regressor = Series(data=zeros(len(DATE_RANGES[ANALYSIS_YEAR])),index=DATE_RANGES[ANALYSIS_YEAR])
hurricane_regressor = Series(data=zeros(len(DATE_RANGES[ANALYSIS_YEAR])),index=DATE_RANGES[ANALYSIS_YEAR])

In [16]:
# Binary values on dates of hurricanes
for name, dates in hurricanes.items():
    for date in dates:
        hurricane_regressor.loc[date] = 1

In [17]:
hurricane_regressor.sort_index(inplace = True)

In [18]:
list(hurricane_regressor.index)

[Timestamp('2012-01-01 00:00:00', freq='D'),
 Timestamp('2012-01-02 00:00:00', freq='D'),
 Timestamp('2012-01-03 00:00:00', freq='D'),
 Timestamp('2012-01-04 00:00:00', freq='D'),
 Timestamp('2012-01-05 00:00:00', freq='D'),
 Timestamp('2012-01-06 00:00:00', freq='D'),
 Timestamp('2012-01-07 00:00:00', freq='D'),
 Timestamp('2012-01-08 00:00:00', freq='D'),
 Timestamp('2012-01-09 00:00:00', freq='D'),
 Timestamp('2012-01-10 00:00:00', freq='D'),
 Timestamp('2012-01-11 00:00:00', freq='D'),
 Timestamp('2012-01-12 00:00:00', freq='D'),
 Timestamp('2012-01-13 00:00:00', freq='D'),
 Timestamp('2012-01-14 00:00:00', freq='D'),
 Timestamp('2012-01-15 00:00:00', freq='D'),
 Timestamp('2012-01-16 00:00:00', freq='D'),
 Timestamp('2012-01-17 00:00:00', freq='D'),
 Timestamp('2012-01-18 00:00:00', freq='D'),
 Timestamp('2012-01-19 00:00:00', freq='D'),
 Timestamp('2012-01-20 00:00:00', freq='D'),
 Timestamp('2012-01-21 00:00:00', freq='D'),
 Timestamp('2012-01-22 00:00:00', freq='D'),
 Timestamp

#### Days or week, month of year and week of year regressors - WORKING

In [19]:
# Creating regressors to control for the days of the week
days_of_week_regressors = DataFrame(index=analysis_range)

for day_of_week in range(7):
    days_of_week_regressors[DAY_NAMES[day_of_week]] = list(map(int, analysis_range.dayofweek==day_of_week))

month_of_year_regressors = DataFrame(index=analysis_range)
for month_of_year in range(1,13):
    month_of_year_regressors[MONTH_NAMES[month_of_year-1]] = list(map(int, analysis_range.month==month_of_year))

week_of_year_regressors = DataFrame(index=analysis_range)
for week_of_year in range(1,53):
    week_of_year_regressors['WEEK_'+str(week_of_year)] = list(map(int, analysis_range.isocalendar().week==week_of_year))

In [20]:
# print(days_of_week_regressors)
# print(month_of_year_regressors)
# print(week_of_year_regressors)

#### Holiday date regressors - WORKING

In [21]:
# Creating a holiday date regressor

csv_file = open(r"C:\Users\clark\OneDrive - University of Warwick\Diss\2. Analysis\Data\Otto data\Misc\holidays_sent.csv", 'r')
reader = csv.DictReader(csv_file)
holidays = {}

for row in reader:
    holidays[pd.to_datetime(row['date'], errors='raise')] = row['holiday']+'_'+str(pd.to_datetime(row['date'], errors='raise').year)[-2:]

holiday_regressor = Series(index=DATE_RANGES[ANALYSIS_YEAR], data=zeros(len(DATE_RANGES[ANALYSIS_YEAR])))
for date in holiday_regressor.index:
    if(date in holidays.keys()): holiday_regressor[date] = 1

# Creates binary regressors for individual holiday dates - like New Years Day...
ind_holiday_regressors = DataFrame(index=analysis_range)
for holiday_date, name in holidays.items():
    if(holiday_date in analysis_range): 
        if(not '*' in name):
            holiday_name = name.replace(' ', '').replace('\'','').replace('*','2').replace('.', '').upper()
            ind_holiday_regressors[holiday_name] = Series( map(int, analysis_range == holiday_date),index=analysis_range)

for holiday_date, name in holidays.items():
    if(holiday_date in analysis_range): 
        if('*' in name):
            holiday_name = name.replace(' ', '').replace('\'','').replace('*','').replace('.', '').upper()
            ind_holiday_regressors[holiday_name].ix[holiday_date] = 1


#### Payday regressors -  WORKING

In [22]:
# Creating a payday regressor
payday_regressors = DataFrame(index=analysis_range)
payday_regressors['FIRST_OF_MONTH'] = zeros(len(analysis_range))
payday_regressors['FIFTEENTH_OF_MONTH'] = zeros(len(analysis_range))

for date in analysis_range:
    # If the day of the date is the 1st of the month...
    if(date.day == 1):
        # If the date is not a business day or a holiday...
        if((not bool(len(pd.bdate_range(date,date)))) or sum(ind_holiday_regressors.loc[date]) > 0):
            # Iterate through the range of 5 
            for i in range(5):
                # If the date is outside of the range of the year
                if((date-i*date.freq)  not in analysis_range):   break
                if(bool(len(pd.bdate_range(date-i*date.freq,date-i*date.freq)))):
                    payday_regressors['FIRST_OF_MONTH'].loc[date-i*date.freq]  = 1
                    break
        else:
            payday_regressors['FIRST_OF_MONTH'].loc[date]  = 1
            
            
    if(date.day == 15):
        # If the date is not a business day or a holiday...
        if((not bool(len(pd.bdate_range(date,date)))) or sum(ind_holiday_regressors.loc[date]) > 0):
            # Iterate through the range of 5 
            for i in range(5):
                # If the date is outside of the range of the year
                if((date-i*date.freq)  not in analysis_range):   break
                if(bool(len(pd.bdate_range(date-i*date.freq,date-i*date.freq)))):
                    payday_regressors['FIFTEENTH_OF_MONTH'].loc[date-i*date.freq]  = 1
                    break
        else:
            payday_regressors['FIFTEENTH_OF_MONTH'].loc[date]  = 1


## Combine the nuisance variables

In [23]:
zip_df = DataFrame(columns = [], index= days_of_week_regressors.index) 


for col in ind_holiday_regressors.columns: zip_df[col.split('_')[0]] = ind_holiday_regressors[col]
for col in payday_regressors.columns:    zip_df[col] = payday_regressors[col]
for col in days_of_week_regressors.columns:    zip_df[col] = days_of_week_regressors[col]
for col in month_of_year_regressors.columns:    zip_df[col] = month_of_year_regressors[col]


In [24]:
#zip_df.to_csv(r"C:\Users\clark\OneDrive - University of Warwick\Diss\2. Analysis\Data\nuisance_variables.csv")

#### Working - Creating the composite_per_capita 

##### This is only needed if we look at 2013 data 

In [25]:
# Creating the "composite_per_capita" metric
lott['composite'] = (lott['Take5'] + lott['Win4Day'] + lott['Win4Eve'] +lott['QuickDraw'] + \
                         lott['Pick10'] +lott['NumbersDay'] + lott['NumbersEve'] ) #/ lott['RetailerCount']


lott['composite_per_capita'] = zeros(lott.shape[0])
grouped = lott.groupby('ZIP')
for zip, zip_lott in grouped:
#     print(zip)
    comp_value = lott.loc[lott.ZIP==zip,'composite'] / demographics[demographics["ZIP"]== zip]["population_over_18"].values[0]
    lott.loc[lott.ZIP==zip,'composite_per_capita'] = comp_value



if(ANALYSIS_YEAR == 2012):
    for zip in lott.ZIP.unique():
        try:
            #print(zip)
            zip_df = lott[lott.ZIP == zip]
            pre_storm_mean = mean(zip_df[:"27/10/2012"].composite_per_capita)

            storm_mean = mean(zip_df[datetime(2012, 11,1).strftime("%d/%m/%Y"):datetime(2012, 12,31).strftime("%d/%m/%Y")].composite_per_capita)
            if(storm_mean/pre_storm_mean < .75):
                print('DROPPING FOR SANDY:', zip)
                lott = lott[lott.ZIP != zip]
        except Exception as e:
            print("There was an error with ", zip)


DROPPING FOR SANDY: 10004
DROPPING FOR SANDY: 10005
DROPPING FOR SANDY: 10006
DROPPING FOR SANDY: 11414
DROPPING FOR SANDY: 10038
There was an error with  10044
DROPPING FOR SANDY: 10280
DROPPING FOR SANDY: 11224
There was an error with  11691
There was an error with  11692
There was an error with  11693
There was an error with  11694
There was an error with  11697


In [26]:
lott

,ZIP,Lotto,NumbersEve,Win4Eve,Pick10,Take5,QuickDraw,NumbersDay,Win4Day,MegaMillions,Megaplier,SweetMillion,PowerBall,PowerPlay,RetailerCount,composite,composite_per_capita
SalesDate,,,,,,,,,,,,,,,,,
01/01/2012,10001,315,3410.0,2652.5,223,1336,1087,2032.5,1700.5,654,27,213,492,30,31,12441.5,0.727360
02/01/2012,10001,507,4599.0,4606.0,401,2135,1285,4063.5,2992.5,1907,34,1295,1598,68,51,20082.0,1.174043
03/01/2012,10001,1576,9717.0,7769.5,762,4172,1551,9403.0,6600.0,12289,372,674,4376,187,70,39974.5,2.337007
04/01/2012,10001,3159,9392.5,8316.0,661,3550,2427,8797.0,6812.5,2538,86,556,6705,474,70,39956.0,2.335925
05/01/2012,10001,1254,11006.0,9133.5,831,4251,1276,10518.5,7796.0,4512,98,3383,3264,141,70,44812.0,2.619819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27/12/2012,11234,833,10984.0,12201.5,587,5501,8240,8350.5,8434.0,2009,47,1748,1964,34,83,54298.0,0.803915
28/12/2012,11234,1682,12838.5,14628.5,724,7196,4240,9077.0,9893.5,10425,547,208,4777,203,84,58597.5,0.867571
29/12/2012,11234,3732,11768.5,13282.5,730,6539,4037,9745.0,9799.0,1295,30,191,8274,404,83,55901.0,0.827648


#### Splitting the sales into counties

In [27]:
# Splitting the sales and Zips into counties
print(len(unique(lott.ZIP)), 'ZIPs total')
zip_to_county = DataFrame(columns=['ZIP', 'county'])


zip_to_county['county'] = [(x.split(':')[0]) for x in demographics.desc]

zip_to_county['ZIP'] = demographics.index

zip_to_county
lott['date'] = lott.index
lott = lott.merge(zip_to_county, on='ZIP')


166 ZIPs total


In [28]:
lott

,ZIP,Lotto,NumbersEve,Win4Eve,Pick10,Take5,QuickDraw,NumbersDay,Win4Day,MegaMillions,Megaplier,SweetMillion,PowerBall,PowerPlay,RetailerCount,composite,composite_per_capita,date,county
0,10001,315,3410.0,2652.5,223,1336,1087,2032.5,1700.5,654,27,213,492,30,31,12441.5,0.727360,01/01/2012,Manhattan
1,10001,507,4599.0,4606.0,401,2135,1285,4063.5,2992.5,1907,34,1295,1598,68,51,20082.0,1.174043,02/01/2012,Manhattan
2,10001,1576,9717.0,7769.5,762,4172,1551,9403.0,6600.0,12289,372,674,4376,187,70,39974.5,2.337007,03/01/2012,Manhattan
3,10001,3159,9392.5,8316.0,661,3550,2427,8797.0,6812.5,2538,86,556,6705,474,70,39956.0,2.335925,04/01/2012,Manhattan
4,10001,1254,11006.0,9133.5,831,4251,1276,10518.5,7796.0,4512,98,3383,3264,141,70,44812.0,2.619819,05/01/2012,Manhattan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60649,11234,833,10984.0,12201.5,587,5501,8240,8350.5,8434.0,2009,47,1748,1964,34,83,54298.0,0.803915,27/12/2012,Brooklyn
60650,11234,1682,12838.5,14628.5,724,7196,4240,9077.0,9893.5,10425,547,208,4777,203,84,58597.5,0.867571,28/12/2012,Brooklyn
60651,11234,3732,11768.5,13282.5,730,6539,4037,9745.0,9799.0,1295,30,191,8274,404,83,55901.0,0.827648,29/12/2012,Brooklyn
60652,11234,757,11238.5,11494.0,628,5503,3746,8315.0,7689.0,1201,99,283,1308,80,82,48613.5,0.719752,30/12/2012,Brooklyn


### : Otto et al formatting data for regressions.

In [29]:
grouped = lott.groupby('ZIP')
for zip, zip_lott in grouped:
    print(zip)
    print(zip_lott)

10001
       ZIP  Lotto  NumbersEve  Win4Eve  Pick10  Take5  QuickDraw  NumbersDay  \
0    10001    315      3410.0   2652.5     223   1336       1087      2032.5   
1    10001    507      4599.0   4606.0     401   2135       1285      4063.5   
2    10001   1576      9717.0   7769.5     762   4172       1551      9403.0   
3    10001   3159      9392.5   8316.0     661   3550       2427      8797.0   
4    10001   1254     11006.0   9133.5     831   4251       1276     10518.5   
..     ...    ...         ...      ...     ...    ...        ...         ...   
361  10001   1121      9632.5   8976.5     585   5118       4163      9392.5   
362  10001   2118     10908.5  10368.5     984   6298       3385     10554.5   
363  10001   1997      5751.0   5191.0     614   2927       1402      5379.0   
364  10001    407      3809.5   3597.0     255   2032        383      2739.5   
365  10001   1210      7839.5   8408.5    1078   5608       2463      7936.0   

     Win4Day  MegaMillions  Megap

         ZIP  Lotto  NumbersEve  Win4Eve  Pick10  Take5  QuickDraw  \
12070  10014    196      2018.0   1357.5     122    818          0   
12071  10014    313      2269.5   1820.0     155   1344        166   
12072  10014    572      3891.5   3514.0     195   1907         94   
12073  10014   1116      3665.0   3167.5     204   1703         97   
12074  10014    395      3958.0   3560.0     286   2021         92   
...      ...    ...         ...      ...     ...    ...        ...   
12429  10014    301      3250.0   4016.5     225   2971         76   
12430  10014    606      4199.5   4924.5     163   2892         52   
12431  10014   1034      2281.5   2271.5     220   1979          0   
12432  10014    201      1838.0   1789.0     183   1675          0   
12433  10014    522      3110.0   3291.0     148   2948          0   

       NumbersDay  Win4Day  MegaMillions  Megaplier  SweetMillion  PowerBall  \
12070      1132.0    798.5           374          8            95        224   

         ZIP  Lotto  NumbersEve  Win4Eve  Pick10  Take5  QuickDraw  \
22678  10025    448      6215.5   5586.0    1209   2842         20   
22679  10025    446      8518.0   7122.5     537   3431        469   
22680  10025    908      8539.0   8455.0     914   4164        127   
22681  10025   2032      8474.0   7666.0     504   3637        446   
22682  10025    600      9700.0   8729.5     434   3839        234   
...      ...    ...         ...      ...     ...    ...        ...   
23039  10025    558      7786.0   8188.0     427   4067       1599   
23040  10025   1058      8607.5   8735.5     745   4983        598   
23041  10025   2343      8227.0   7959.5     456   4675        988   
23042  10025    371      6479.5   6887.5     565   3801        892   
23043  10025    706      8733.0   8981.0     451   5050        827   

       NumbersDay  Win4Day  MegaMillions  Megaplier  SweetMillion  PowerBall  \
22678      4679.0   3839.0           870         12           443        565   

         ZIP  Lotto  NumbersEve  Win4Eve  Pick10  Take5  QuickDraw  \
37318  10039     32      2590.5   2413.0     107    725          0   
37319  10039     61      3524.5   3685.0     142   1103          0   
37320  10039    175      3871.0   4065.5     232   1152          0   
37321  10039    504      4170.0   3933.0     210   1046          0   
37322  10039    108      4011.5   4105.5     150   1228          0   
...      ...    ...         ...      ...     ...    ...        ...   
37679  10039     63      3680.5   3895.5     145   1142          0   
37680  10039    222      3921.0   3874.0     166   1243          0   
37681  10039    591      4115.5   4247.0     148   1275          0   
37682  10039     65      2976.0   3296.0     159   1129          0   
37683  10039    200      4474.0   4825.0     256   1684          0   

       NumbersDay  Win4Day  MegaMillions  Megaplier  SweetMillion  PowerBall  \
37318      1906.5   1676.5            88          9            65         52   

         ZIP  Lotto  NumbersEve  Win4Eve  Pick10  Take5  QuickDraw  \
51216  10309    176      2529.5   1819.5     149   1584       3116   
51217  10309    481      3474.5   3111.5     397   2814       3413   
51218  10309    627      3724.0   2693.5     391   2006       5278   
51219  10309   1311      3519.5   2261.5     241   1868       8262   
51220  10309    326      3760.0   2771.5     129   2160       5700   
...      ...    ...         ...      ...     ...    ...        ...   
51577  10309    346      4073.5   3219.0     249   2087      10517   
51578  10309    788      4848.0   4559.5     675   2991       7031   
51579  10309   1459      4788.5   4580.5     256   2313       8548   
51580  10309    338      3398.5   3063.5     125   1673       4696   
51581  10309    593      4791.5   3798.5     184   2744       3925   

       NumbersDay  Win4Day  MegaMillions  Megaplier  SweetMillion  PowerBall  \
51216      1670.0    797.5           319          3            99        301   

...

         ZIP  Lotto  NumbersEve  Win4Eve  Pick10  Take5  QuickDraw  \
57777  11231     93      3023.5   2158.0      76   1009          0   
57778  11231    114      5842.0   2861.5     150   1329        617   
57779  11231    305      4528.0   3314.5     124   1384        320   
57780  11231    755      4647.0   2971.0     165   1527        983   
57781  11231    177      4824.5   2832.5     126   1391        715   
...      ...    ...         ...      ...     ...    ...        ...   
58138  11231    159      4013.0   3200.5      95   1285       1338   
58139  11231    370      4615.5   4269.0     123   1782       1325   
58140  11231    742      5933.5   5309.0     136   1574       2078   
58141  11231    132      3364.0   2982.0     104   1260        213   
58142  11231    271      8317.0   4988.0     168   1759        626   

       NumbersDay  Win4Day  MegaMillions  Megaplier  SweetMillion  PowerBall  \
57777      2001.5   1228.0           204          6            67        119   

        ZIP  Lotto  NumbersEve  Win4Eve  Pick10  Take5  QuickDraw  NumbersDay  \
8413  11357    334      1693.5   1237.5     123   1390        707       979.5   
8414  11357    464      2287.5   1636.0     112   1915        799      1307.5   
8415  11357    723      1781.5   1474.5     147   1926        660      1008.0   
8416  11357   1157      2115.5   1569.0      91   1608        410      1232.0   
8417  11357    470      2071.5   1453.0     190   1693        321      1231.5   
...     ...    ...         ...      ...     ...    ...        ...         ...   
8774  11357    417      1908.5   1390.5     147   1966       1327      1305.5   
8775  11357    682      1994.5   1303.5     135   2461        687      1390.0   
8776  11357   1386      2012.0   1678.0     134   1910        722      1209.0   
8777  11357    493      1781.0   1307.5     169   1970       2016      1218.5   
8778  11357    652      2414.0   1821.5     231   2960       1394      1444.5   

      Win4Day  MegaMillions

         ZIP  Lotto  NumbersEve  Win4Eve  Pick10  Take5  QuickDraw  \
24508  11374    232      1799.0   1425.5     148   1286       2059   
24509  11374    288      2203.5   1795.0     168   1507        563   
24510  11374    438      2165.5   2046.5     136   1503        634   
24511  11374    977      2217.0   2044.5     116   1301       1244   
24512  11374    347      2451.0   2194.5     126   1446       1669   
...      ...    ...         ...      ...     ...    ...        ...   
24869  11374    321      1999.5   2592.5     184   1299       4767   
24870  11374    578      2197.0   2677.0     183   2120       1580   
24871  11374   1333      2371.5   2983.0     214   1881       2507   
24872  11374    297      1660.5   2172.0     166   1405       2466   
24873  11374    475      2453.0   2867.5     267   2219       4024   

       NumbersDay  Win4Day  MegaMillions  Megaplier  SweetMillion  PowerBall  \
24508      1471.0   1023.5           560         12           203        373   

         ZIP  Lotto  NumbersEve  Win4Eve  Pick10  Take5  QuickDraw  \
36952  11417    106      2764.5   2670.5      84   1419       1106   
36953  11417    263      3336.0   2994.0     185   1577        505   
36954  11417    370      2844.0   2773.5     144   1383       2542   
36955  11417    944      3011.0   2747.0     145   1332       6394   
36956  11417    269      3862.5   3341.5      95   1388       3319   
...      ...    ...         ...      ...     ...    ...        ...   
37313  11417    207      2675.5   3249.5     146   1436       2604   
37314  11417    409      3416.0   3611.5     178   2034       2710   
37315  11417   1061      3024.5   3398.5     152   1776       2354   
37316  11417    147      2695.5   3007.0     100   1699       1096   
37317  11417    341      3199.5   3927.0      91   1876       1420   

       NumbersDay  Win4Day  MegaMillions  Megaplier  SweetMillion  PowerBall  \
36952      2093.0   1626.0           232          3           127        230   

[366 rows x 19 columns]
11434
         ZIP  Lotto  NumbersEve  Win4Eve  Pick10  Take5  QuickDraw  \
50850  11434    179      8595.0   8724.0     547   2546        792   
50851  11434    392     11239.5  11942.5     881   3546       1913   
50852  11434    801     11496.5  12180.5     914   4255       1271   
50853  11434   2205     11659.0  12211.5     870   3698       3608   
50854  11434    445     11105.0  12218.0     889   3787       1857   
...      ...    ...         ...      ...     ...    ...        ...   
51211  11434    326     11111.5  12031.5     647   4600       2294   
51212  11434    797     12220.0  13114.5     733   5957       2160   
51213  11434   2520     12488.5  13128.5     614   5376       2841   
51214  11434    224      8860.0  10474.0     461   3750       1358   
51215  11434    615     12840.0  13997.5     818   5501       2991   

       NumbersDay  Win4Day  MegaMillions  Megaplier  SweetMillion  PowerBall  \
50850      6624.0   6126.5           445         

#### Loading in our sentiment scores

In [30]:
# Sent it just our master data!!
sent = read_csv(r'C:\Users\clark\OneDrive - University of Warwick\Diss\2. Analysis\Data\master_data_no_neg_pysent.csv', index_col = "date")

sent.drop(['Unnamed: 0'], axis = 1, inplace = True)

vader_cols =list(sent.columns[sent.columns.str.contains("vader")])
pysent_cols = list(sent.columns[sent.columns.str.contains("pysent")])
roberta_cols = list(list(sent.columns[sent.columns.str.contains("mean_roberta")]))
distilbert_cols = list(list(sent.columns[sent.columns.str.contains("mean_distilbert")]))
no_neg_cols = list(list(sent.columns[sent.columns.str.contains("no_neg_")]))

sent_cols = vader_cols + pysent_cols + roberta_cols + distilbert_cols + no_neg_cols
# Select the columns from sent that have our sentiment analysers
sent = sent.loc[:,sent_cols]

In [31]:
# Set each of the indexes to type date time 
sent.index = to_datetime(sent.index)
lott.index = to_datetime(lott.date)
lott = lott.drop("date", axis = 1)

#### Mergring lott and sent together

In [32]:
lott_sent = lott.merge(sent, left_index = True, right_index = True)
# lott_sent.index = lott_sent.date
#lott_sent

#### Here we have replaced any missing rows in the lott_sent dataframe with the row from the prevous date. Will need to know how many of these dates this occurred on 

In [33]:
# Date range to iterate through to check for missing dates
date_range = pd.date_range(datetime(2012, 1, 1), datetime(2012, 12,31))

# Dataframe to add to 
lott_sent_all_dates = pd.DataFrame(columns =list(lott_sent.columns) )

# Total dates updated
total_dates_changed = 0

# Creates an iterable of grouped df by ZIP
grouped = lott_sent.groupby('ZIP') 

for zip, zip_lott in grouped:
    
    # Create a date range for the year 2012
    date_range = pd.date_range(datetime(2012, 1, 1), datetime(2012, 12,31))
    
    if len(zip_lott)!= 366:
        print(zip, str(len(zip_lott)))
    # Check if every date is in the zip_lott date column
    for date_ in date_range:
        
        # Update total_dates_changed
        total_dates_changed += 1
        
        if date_ not in zip_lott.index:
            print(date_)
            if date_ != datetime(2012, 1, 1):
                # if date not in date_range then duplicate the row from the date before
                # row will be the same as the prior day
                prior_date = date_ - timedelta(days=1)
              
                zip_lott.loc[date_] = zip_lott.loc[prior_date]
                #print("The length once processed is :", str(len(zip_lott)))
            
            else:
                next_date = date_ + timedelta(days=1)
                zip_lott.loc[date_] = zip_lott.loc[next_date]
    # Set the index to the date and sort
    zip_lott.sort_index(axis = "index")
    
    
    lott_sent_all_dates = pd.concat([lott_sent_all_dates, zip_lott])


10002 365
2012-10-31 00:00:00
10007 365
2012-10-30 00:00:00
10009 365
2012-10-30 00:00:00
10010 364
2012-10-30 00:00:00
2012-10-31 00:00:00
10012 365
2012-10-31 00:00:00
10013 364
2012-10-30 00:00:00
2012-10-31 00:00:00
10014 364
2012-02-11 00:00:00
2012-10-30 00:00:00
10044 358
2012-01-11 00:00:00
2012-02-11 00:00:00
2012-03-11 00:00:00
2012-04-11 00:00:00
2012-05-11 00:00:00
2012-07-11 00:00:00
2012-10-30 00:00:00
2012-10-31 00:00:00
10307 364
2012-10-30 00:00:00
2012-10-31 00:00:00
11005 362
2012-01-01 00:00:00
2012-08-04 00:00:00
2012-09-17 00:00:00
2012-09-26 00:00:00
11691 363
2012-01-11 00:00:00
2012-03-11 00:00:00
2012-10-30 00:00:00
11692 357
2012-01-11 00:00:00
2012-03-11 00:00:00
2012-04-11 00:00:00
2012-05-11 00:00:00
2012-06-11 00:00:00
2012-09-11 00:00:00
2012-10-11 00:00:00
2012-10-30 00:00:00
2012-11-11 00:00:00
11693 355
2012-01-11 00:00:00
2012-02-11 00:00:00
2012-03-11 00:00:00
2012-05-11 00:00:00
2012-10-11 00:00:00
2012-10-30 00:00:00
2012-11-11 00:00:00
2012-11-13

#### Check that missing dates have been included

In [34]:
# Investigating 2012-10-31 for 10002 - working
# lott_sent_all_dates_10002 = lott_sent_all_dates[lott_sent_all_dates["ZIP"] == 10002]
# lott_sent_all_dates_10002.loc[["2012-10-30 00:00:00","2012-10-31 00:00:00"]]

In [35]:
# Are there any missing dates in lott_sent_all_dates?
grouped_2 = lott_sent_all_dates.groupby("ZIP")

for zip, zip_lott in grouped_2:
    
    # Create a date range for the year 2012
    date_range = pd.date_range(datetime(2012, 1, 1), datetime(2012, 12,31))
    
    if len(zip_lott)!= 366:
        print(zip, str(len(zip_lott)))
    # Check if every date is in the zip_lott date column
    for date_ in date_range:
        if date_ not in zip_lott.index:
            print(zip, date_)

#### Assign lott_sent_all_dates -> lott_sent (to be used in next step - outputting regressors df) 

In [36]:
####### ASSSIGN TO lott_sent
lott_sent = lott_sent_all_dates 

#### Create the lott_sent_regress_df that can be outputted to a csv for R analysis

In [37]:
# Create an empty df
lott_sent_regress_df = DataFrame() 


# Creates an iterable of grouped df by ZIP
grouped = lott_sent.groupby('ZIP') 


# List of sentiment colums
sent_cols = vader_cols + pysent_cols + roberta_cols + distilbert_cols + no_neg_cols

for zip, zip_lott in grouped:
    print('***', zip, demographics.loc[demographics.ZIP ==zip,'desc'])  #, len(sent[sent.county==zip_lott.county[0]].affect_mean), 'affect days'

    lott_sent.loc[lott_sent.ZIP==zip,'composite_per_capita'] = lott_sent.loc[lott_sent.ZIP==zip,'composite'] / demographics[demographics["ZIP"]== zip]["population_over_18"].values[0]
    zip_df = DataFrame(columns = [], index=zip_lott.index) 
    zip_df['ZIP'] = repeat(zip, len(zip_lott))
    zip_df['z_ses'] = repeat(demographics.loc[demographics.ZIP == zip, 'ses'].values, len(zip_lott))
    zip_df['county'] = zip_lott['county']
    zip_df['purchase'] = zip_lott['composite'] /  demographics[demographics["ZIP"]== zip]["population_over_18"].values[0]
    zip_df['log_purchase'] = log(zip_lott['composite'] /  demographics[demographics["ZIP"]== zip]["population_over_18"].values[0] )
    #zip_df['affect_mean'] = pd_zscore(zip_lott['affect_mean'])
    #zip_df['STORM'] = storm_regressor
    zip_df['HURRICANE'] = hurricane_regressor
    zip_df['HOLIDAYS'] = holiday_regressor    
    #zip_df['BAD_WEATHER'] = storm_regressor + hurricane_regressor
    

    for col in ind_holiday_regressors.columns: zip_df[col.split('_')[0]] = ind_holiday_regressors[col]
    for col in payday_regressors.columns:    zip_df[col] = payday_regressors[col]
    for col in days_of_week_regressors.columns:    zip_df[col] = days_of_week_regressors[col]
    for col in month_of_year_regressors.columns:    zip_df[col] = month_of_year_regressors[col]

    if((sum(isinf(zip_df['log_purchase'])) > 0) or (sum(isnan(zip_df['log_purchase'])) > 0)):
        print('\t missing' ,sum(isinf(zip_df['log_purchase'])) + sum(isnan(zip_df['log_purchase'])), 'observations')
    
#     try:
#         if(ANALYSIS_YEAR == 2012):
#             zip_df = concat([ zip_df.loc[pd.date_range(datetime(2012,1,1).strftime("%d/%m/%Y"),datetime(2012,10,27).strftime("%d/%m/%Y"))],
#                               zip_df.loc[pd.date_range(datetime(2012,11,4).strftime("%d/%m/%Y"),datetime(2012,12,31).strftime("%d/%m/%Y"))]])
#     except Exception as e:
#         print("Missing data in ", zip, e)
        

    for col in zip_df.columns:
        if(zip_df[col].dtype != dtype('O')):
            assert(isnan(zip_df[col]).all() == False)
    #print(zip_df.index)
    
    
    
    

    lott_sent_regress_df = concat([lott_sent_regress_df, zip_df])  

# Remove any rows where the log_purchase is infinitity or zero
lott_sent_regress_df = lott_sent_regress_df[~isinf(lott_sent_regress_df['log_purchase'])]
lott_sent_regress_df = lott_sent_regress_df[~isnan(lott_sent_regress_df['log_purchase'])]

### ADD IN THE SENTIMENT SCORES FOR EACH DAY
# Set the index of sent = date so that we can merge it with lott_sent_regress_df
lott_sent_regress_df = lott_sent_regress_df.merge(sent, left_index = True, right_index = True)

# Save the file down
lott_sent_regress_df.to_csv(r'C:\Users\clark\OneDrive - University of Warwick\Diss\2. Analysis\Data\otto_regression_input_no_neg_pysent.csv')



*** 10001 10001    Manhattan:Chelsea and Clinton
Name: desc, dtype: object
*** 10002 10002    Manhattan:Lower East Side
Name: desc, dtype: object


C:\Users\clark\miniconda3\lib\site-packages\pandas\core\arraylike.py:274: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\clark\miniconda3\lib\site-packages\pandas\core\arraylike.py:274: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


	 missing 3 observations
*** 10003 10003    Manhattan:Lower East Side
Name: desc, dtype: object
	 missing 3 observations
*** 10007 10007    Manhattan:Lower Manhattan
Name: desc, dtype: object


C:\Users\clark\miniconda3\lib\site-packages\pandas\core\arraylike.py:274: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


	 missing 2 observations
*** 10009 10009    Manhattan:Lower East Side
Name: desc, dtype: object


...

C:\Users\clark\miniconda3\lib\site-packages\pandas\core\arraylike.py:274: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


*** 11694 11694    Queens:Rockaways
Name: desc, dtype: object
	 missing 6 observations


C:\Users\clark\miniconda3\lib\site-packages\pandas\core\arraylike.py:274: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


*** 11697 11697    Queens:Rockaways
Name: desc, dtype: object
	 missing 6 observations


C:\Users\clark\miniconda3\lib\site-packages\pandas\core\arraylike.py:274: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
